In [1]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()

genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

In [3]:
# Function Calling Basic
import google.generativeai as genai


def get_price(product: str) -> int:
    """제품의 가격을 알려주는 함수

    Args:
        theme: 제품명
    """
    return 1000


def get_temperature(city: str) -> float:
    """도시의 온도를 알려주는 함수

    Args:
        genre: 도시명
    """
    return 20.5


model = genai.GenerativeModel(model_name="gemini-2.0-flash", tools=[get_price, get_temperature])
# auto function calling(enable_automatic_function_calling=True)
# Manually calling functions(false option) is often preferred over than auto calling because the multiton chat can occasionally malfunction and not operate correctly
# using function calling entails more token spending*
chat_session = model.start_chat(enable_automatic_function_calling=True)
response = chat_session.send_message("서울의 온도는?")
print(response.text)

서울의 온도는 20.5도입니다.


In [4]:
prod_database = {
    "갤럭시 S24": {"재고": 10, "가격": 1_700_000},
    "갤럭시 S23": {"재고": 5, "가격": 1_300_000},
    "갤럭시 S22": {"재고": 3, "가격": 1_100_000},
}


def is_product_available(product_name: str) -> bool:
    """특정 제품의 재고가 있는지 확인한다.

    Args:
        product_name: 제품명
    """
    if product_name in prod_database:
        if prod_database[product_name]["재고"] > 0:
            return True
    return False


def get_product_price(product_name: str) -> int:
    """제품의 가격을 가져온다.

    Args:
        product_name: 제품명
    """
    if product_name in prod_database:
        return prod_database[product_name]["가격"]
    return None


def place_order(product_name: str, address: str) -> str:
    """제품 주문결과를 반환한다.
    Args:
        product_name: 제품명
        address: 배송지
    """
    if is_product_available(product_name):
        prod_database[product_name]["재고"] -= 1
        return "주문 완료"
    else:
        return "재고 부족으로 주문 불가"

In [38]:
# function mapping info
function_repoistory = {
    "is_product_available": is_product_available,
    "get_product_price": get_product_price,
    "place_order": place_order
}

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    tools=function_repoistory.values()
)


def correct_response(response):
    part = response.candidates[0].content.parts[0]
    if part.function_call:
        for k, v in part.function_call.args.items():
            byte_v = bytes(v, "utf-8").decode("unicode_escape")
            corrected_v = bytes(byte_v, "latin1").decode("utf-8")
            part.function_call.args.update({k: corrected_v})

In [39]:
chat_session = model.start_chat()
prompt = "갤럭시 S24 판매 중인가요?"
response = chat_session.send_message(prompt)
correct_response(response)
print(response.candidates[0].content)

parts {
  function_call {
    name: "is_product_available"
    args {
      fields {
        key: "product_name"
        value {
          string_value: "\352\260\244\353\237\255\354\213\234 S24"
        }
      }
    }
  }
}
role: "model"


In [34]:
response.candidates[0].content.parts[0]

function_call {
  name: "is_product_available"
  args {
    fields {
      key: "product_name"
      value {
        string_value: "\352\260\244\353\237\255\354\213\234 S24"
      }
    }
  }
}

In [40]:
# to reuse 
part = response.candidates[0].content.parts[0]
if part.function_call:
    function_call = part.function_call
    function_name = function_call.name
    function_args = {k: v for k, v in function_call.args.items()}
    print(f"{function_name} args=>: {function_args}")
    function_result = function_repoistory[function_name](**function_args)
    print(f"{function_name} result=>: {function_result}")

is_product_available args=>: {'product_name': '갤럭시 S24'}
is_product_available result=>: True


In [41]:
import google.ai.generativelanguage as glm

part = glm.Part(
    function_response=glm.FunctionResponse(
        name="is_product_available",
        response={
            "content": function_result,
        },
    )
)
print(part)
response = chat_session.send_message(part)
correct_response(response)
print("-" * 50)
print(response.candidates[0].content)

function_response {
  name: "is_product_available"
  response {
    fields {
      key: "content"
      value {
        bool_value: true
      }
    }
  }
}
--------------------------------------------------
parts {
  text: "\352\260\244\353\237\255\354\213\234 S24\353\212\224 \355\230\204\354\236\254 \355\214\220\353\247\244 \354\244\221\354\236\205\353\213\210\353\213\244.\n"
}
role: "model"


In [45]:
#  스마트폰 주문 챗봇 구현
prod_database = {
    "갤럭시 S24": {"재고": 10, "가격": 1_700_000},
    "갤럭시 S23": {"재고": 5, "가격": 1_300_000},
    "갤럭시 S22": {"재고": 3, "가격": 1_100_000},
}


def is_product_available(product_name: str) -> bool:
    """특정 제품의 재고가 있는지 확인한다.

    Args:
        product_name: 제품명
    """

    if product_name in prod_database:
        if prod_database[product_name]["재고"] > 0:
            return True
    return False


def get_product_price(product_name: str) -> int:
    """제품의 가격을 가져온다.

    Args:
        product_name: 제품명
    """
    if product_name in prod_database:
        return prod_database[product_name]["가격"]
    return None


def place_order(product_name: str, address: str) -> str:
    """제품 주문결과를 반환한다.
    Args:
        product_name: 제품명
        address: 배송지
    """
    if is_product_available(product_name):
        prod_database[product_name]["재고"] -= 1
        return "주문 완료"
    else:
        return "재고 부족으로 주문 불가"


function_repoistory = {
    "is_product_available": is_product_available,
    "get_product_price": get_product_price,
    "place_order": place_order
}


# response character set change from unicode_escape to utf-8
def correct_response(response):
    # print(f'correct_response : {response}')
    part = response.candidates[0].content.parts[0]
    if part.function_call:
        for k, v in part.function_call.args.items():
            byte_v = bytes(v, "utf-8").decode("unicode_escape")
            corrected_v = bytes(byte_v, "latin1").decode("utf-8")
            part.function_call.args.update({k: corrected_v})


model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    tools=function_repoistory.values()
)

In [48]:
chat_session = model.start_chat(history=[])
# 이 방식은 혹시라도 사용자가 갤럭시S24 라고 붙여서 질의 할 경우에는 작동 안함. 
queries = ["갤럭시 S24 판매 중인가요?", "가격은 어떻게 되나요?", "서울시 종로구 종로1가 1번지로 배송부탁드립니다"]

for query in queries:
    print(f"\n사용자: {query}")
    response = chat_session.send_message(query)
    correct_response(response)
    part = response.candidates[0].content.parts[0]

    if part.function_call:
        function_call = part.function_call
        function_name = function_call.name
        function_args = {k: v for k, v in function_call.args.items()}
        function_result = function_repoistory[function_name](**function_args)
        part = glm.Part(
            function_response=glm.FunctionResponse(
                name=function_name,
                response={"content": function_result, },
            )
        )
        response = chat_session.send_message(part)
        print(f"모델: {response.candidates[0].content.parts[0].text}")


사용자: 갤럭시 S24 판매 중인가요?
correct_response : response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "is_product_available",
                  "args": {
                    "product_name": "\uac24\ub7ed\uc2dc S24"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP"
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 132,
        "candidates_token_count": 14,
        "total_token_count": 146
      }
    }),
)
모델: 갤럭시 S24 판매 중입니다.


사용자: 가격은 어떻게 되나요?
correct_response : response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
             

In [44]:
print(f"history:\n{chat_session.history}")

history:
[parts {
  text: "\352\260\244\353\237\255\354\213\234 S24 \355\214\220\353\247\244 \354\244\221\354\235\270\352\260\200\354\232\224?"
}
role: "user"
, parts {
  function_call {
    name: "is_product_available"
    args {
      fields {
        key: "product_name"
        value {
          string_value: "\352\260\244\353\237\255\354\213\234 S24"
        }
      }
    }
  }
}
role: "model"
, parts {
  function_response {
    name: "is_product_available"
    response {
      fields {
        key: "content"
        value {
          bool_value: true
        }
      }
    }
  }
}
role: "user"
, parts {
  text: "\352\260\244\353\237\255\354\213\234 S24\353\212\224 \355\230\204\354\236\254 \355\214\220\353\247\244 \354\244\221\354\236\205\353\213\210\353\213\244.\n"
}
role: "model"
, parts {
  text: "\352\260\200\352\262\251\354\235\200 \354\226\264\353\226\273\352\262\214 \353\220\230\353\202\230\354\232\224?"
}
role: "user"
, parts {
  function_call {
    name: "get_product_price"